In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader, TensorDataset
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.autograd import Variable as V

import random

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class dice_bce_loss(nn.Module):
    def __init__(self, batch=True):
        super(dice_bce_loss, self).__init__()
        self.batch = batch
        self.bce_loss = nn.BCEWithLogitsLoss()

    def soft_dice_coeff(self, y_true, y_pred):
        smooth = 0.0  # may change
        if self.batch:
            i = torch.sum(y_true)
            j = torch.sum(y_pred)
            intersection = torch.sum(y_true * y_pred)
        else:
            i = y_true.sum(1).sum(1).sum(1)
            j = y_pred.sum(1).sum(1).sum(1)
            intersection = (y_true * y_pred).sum(1).sum(1).sum(1)
        score = (2. * intersection + smooth) / (i + j + smooth)
        return score.mean()

    def soft_dice_loss(self, y_true, y_pred):
        loss = 1 - self.soft_dice_coeff(y_true, y_pred)
        return loss

    def __call__(self, y_true, y_pred):
        a =  self.bce_loss(y_pred, y_true)
        b =  self.soft_dice_loss(y_true, y_pred)
        return a + b


In [5]:
import torch
import torch.nn as nn

from torch.autograd import Variable
from torchvision import models
import torch.nn.functional as F
from functools import partial

nonlinearity = partial(F.relu, inplace=True)


#CBAM改

class Channel(nn.Module ):
    def __init__(self,inchannel,reduction=16):
        super(Channel, self).__init__()
        radio=reduction //2


        self.gap=nn.AdaptiveAvgPool2d(1)
        self.gmp=nn.AdaptiveMaxPool2d (1)

        self.mlp=nn.Sequential (
            nn.Conv2d(inchannel ,inchannel //radio ,1,1,0,bias= False ),
            nn.ReLU (),
            nn.Conv2d (inchannel //radio,inchannel//reduction ,1,1,0,bias= False ),
            nn.ReLU (),
            nn.Conv2d (inchannel //reduction ,inchannel//radio ,1,1,0,bias= False ),
            nn.ReLU (),
            nn.Conv2d (inchannel//radio,inchannel ,1,1,0,bias= False )
        )
        self.sig=nn.Sigmoid ()

    def forward(self,x):

        gap=self.gap (x)
        gmp=self.gmp(x)

        gap=self.mlp(gap)
        gmp=self.mlp(gmp)


        return self.sig(gap+gmp)


class Spatial(nn.Module ):
    def __init__(self):
        super(Spatial, self).__init__()

        self.conv=nn.Sequential (

            nn.Conv2d (in_channels= 2,out_channels= 2,kernel_size= 3,stride= 1,padding= 1,bias= False ),
            nn.ReLU (),
            nn.Conv2d (in_channels= 2,out_channels= 2,kernel_size= 3,stride= 1,padding= 1,bias= False ),
            nn.ReLU (),
            nn.Conv2d (in_channels= 2,out_channels= 1,kernel_size= 3,stride= 1,padding= 1,bias= False )
        )

        self.sig=nn.Sigmoid ()

    def forward(self,x):

        max_pool,_=torch.max (x,dim=1,keepdim= True)
        avg_pool  =torch.mean(x,dim=1,keepdim= True)

        cc=torch.cat([max_pool ,avg_pool ],dim= 1)

        cc=self.conv (cc)

        cc=self.sig(cc)

        return cc


class CBAMs(nn.Module ):
    def __init__(self,inchannel,reduction):
        super(CBAMs, self).__init__()

        self.channel=Channel (inchannel ,reduction )
        self.spatial=Spatial ()

    def forward(self,x):

        ca=self.channel (x)
        out=ca*x
        sa=self.spatial (out)
        out=out*sa

        return out

class Dblock(nn.Module):
    def __init__(self, channel,d_bin):
        super(Dblock, self).__init__()
        self.dilate1 = nn.Conv2d(channel, channel, kernel_size=3, dilation=d_bin[0], padding=d_bin[0])
        self.dilate2 = nn.Conv2d(channel, channel, kernel_size=3, dilation=d_bin[1], padding=d_bin[1])
        self.dilate3 = nn.Conv2d(channel, channel, kernel_size=3, dilation=d_bin[2], padding=d_bin[2])
        self.dilate4 = nn.Conv2d(channel, channel, kernel_size=3, dilation=d_bin[3], padding=d_bin[3])
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                if m.bias is not None:
                    m.bias.data.zero_()
        self.att=CBAMs(channel,16)

    def forward(self, x):
        dilate1_out = nonlinearity(self.dilate1(x))
        dilate2_out = nonlinearity(self.dilate2(dilate1_out))
        dilate3_out = nonlinearity(self.dilate3(dilate2_out))
        dilate4_out = nonlinearity(self.dilate4(dilate3_out))

        out = x + dilate1_out + dilate2_out + dilate3_out + dilate4_out
        return self.att(out)


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, n_filters):
        super(DecoderBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)
        self.relu1 = nonlinearity

        self.deconv2 = nn.ConvTranspose2d(in_channels // 4, in_channels // 4, 3, stride=2, padding=1, output_padding=1)
        self.norm2 = nn.BatchNorm2d(in_channels // 4)
        self.relu2 = nonlinearity

        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)
        self.relu3 = nonlinearity

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.deconv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        return x
class RFE_LINKNET(nn.Module):
    def __init__(self, num_classes=1, num_channels=3):
        super(RFE_LINKNET, self).__init__()

        filters = [64, 128, 256, 512]
        resnet = models.resnet34(pretrained=True)
        self.firstconv = resnet.conv1
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3
        self.encoder4 = resnet.layer4

        self.dblock1=Dblock(64,d_bin=[1,32,32,64])
        self.dblock2=Dblock(128,d_bin=[1,16,16,32])
        self.dblock3=Dblock(256,d_bin=[1,4,8,16])
        self.dblock4=Dblock(512,d_bin=[1,2,4,8])


        self.decoder4 = DecoderBlock(filters[3], filters[2])
        self.decoder3 = DecoderBlock(filters[2], filters[1])
        self.decoder2 = DecoderBlock(filters[1], filters[0])
        self.decoder1 = DecoderBlock(filters[0], filters[0])



        self.finaldeconv1 = nn.ConvTranspose2d(filters[0], 32, 4, 2, 1)
        self.finalrelu1 = nonlinearity
        self.finalconv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.finalrelu2 = nonlinearity
        self.finalconv3 = nn.Conv2d(32, num_classes, 3, padding=1)

    def forward(self, x):
        # Encoder
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x = self.firstmaxpool(x)
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        # Center

        e4=self.dblock4(e4)

        # Decoder
        d4 = self.decoder4(e4)+self.dblock3(e3)
        d3 = self.decoder3(d4)+self.dblock2(e2)
        d2 = self.decoder2(d3)+self.dblock1(e1)
        d1 = self.decoder1(d2)

        out = self.finaldeconv1(d1)
        out = self.finalrelu1(out)
        out = self.finalconv2(out)
        out = self.finalrelu2(out)
        out = self.finalconv3(out)

        return F.sigmoid(out)



In [7]:
model = RFE_LINKNET(num_classes=1, num_channels=3)  # Adjust parameters if necessary
model.cuda()
checkpoint = torch.load('rfelinknet.pth')
model.load_state_dict(checkpoint['model_state_dict'])



<ipython-input-7-b9d4b125d7fc>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('rfelinknet.pth')


<All keys matched successfully>

In [8]:
!unzip /content/drive/MyDrive/ML_P2_QuantumMinds/test.zip -d data

Archive:  /content/drive/MyDrive/ML_P2_QuantumMinds/test.zip
   creating: data/test/
   creating: data/test/test_set_images/
   creating: data/test/test_set_images/test_1/
  inflating: data/test/test_set_images/test_1/test_1.png  
   creating: data/test/test_set_images/test_10/
  inflating: data/test/test_set_images/test_10/test_10.png  
   creating: data/test/test_set_images/test_11/
  inflating: data/test/test_set_images/test_11/test_11.png  
   creating: data/test/test_set_images/test_12/
  inflating: data/test/test_set_images/test_12/test_12.png  
   creating: data/test/test_set_images/test_13/
  inflating: data/test/test_set_images/test_13/test_13.png  
   creating: data/test/test_set_images/test_14/
  inflating: data/test/test_set_images/test_14/test_14.png  
   creating: data/test/test_set_images/test_15/
  inflating: data/test/test_set_images/test_15/test_15.png  
   creating: data/test/test_set_images/test_16/
  inflating: data/test/test_set_images/test_16/test_16.png  
   cre

In [11]:
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

def extract_patches(input_image, patch_size=384):
    """
    Extracts 4 non-overlapping patches from a 608x608 input image.
    """
    patches = np.empty((4, input_image.shape[2], patch_size, patch_size))
    offsets = [(0, 0), (0, 224), (224, 0), (224, 224)]

    for i, (y, x) in enumerate(offsets):
        patches[i] = np.transpose(input_image[y:y+patch_size, x:x+patch_size, :], (2, 0, 1))

    return patches

def assemble_patches(patches):
    """
    Assembles 4 patches back into a 608x608 output.
    """
    output = np.empty((patches.shape[1], 608, 608))
    offsets = [(0, 0), (0, 224), (224, 0), (224, 224)]

    for i, (y, x) in enumerate(offsets):
        output[:, y:y+384, x:x+384] = patches[i]

    return output

def mask_to_submission_format(mask, index):
    """
    Converts a mask into the submission format.
    """
    submission = []
    for y in range(0, mask.shape[0], 16):
        for x in range(0, mask.shape[1], 16):
            patch = mask[y:y+16, x:x+16]
            prediction = 1 if np.mean(patch > 0.2) > 0.25 else 0
            submission.append([f"{index:03d}_{x}_{y}", prediction])
    return submission

def create_submission(model, test_images_path='test_set_images/', patch_size=384, cuda=True):
    """
    Generates a submission file by predicting masks for the test set images.
    """
    all_submissions = []

    for index in tqdm(range(1, 51)):
        model.eval()

        # Load and preprocess the test image
        input_image = np.array(Image.open(f'{test_images_path}/test_{index}/test_{index}.png')).astype('float32') / 255
        input_patches = extract_patches(input_image)
        input_patches = torch.from_numpy(input_patches).float()

        # Perform inference
        if cuda:
            predictions = model(input_patches.cuda()).detach().cpu().numpy()
        else:
            predictions = model(input_patches).detach().numpy()

        # Assemble patches into a full mask
        output_mask = assemble_patches(predictions)[0]

        if index==19:
            # Save the input image and output mask for index 19
            #input_image_path = os.path.join('', f'input_image_{index}.png')
            output_mask_path = os.path.join('', f'rfe-linknet_test{index}.png')

            # Save input image and output mask
            #Image.fromarray((input_image * 255).astype(np.uint8)).save(input_image_path)
            Image.fromarray((output_mask * 255).astype(np.uint8)).save(output_mask_path)


        # Convert mask to submission format
        submission = mask_to_submission_format(output_mask, index)
        all_submissions.extend(submission)

    # Add header to submission
    submission = [['id', 'prediction']] + all_submissions
    return np.array(submission)

# Example usage
submission = create_submission(model, 'data/test/test_set_images', 384, True)
np.savetxt("submit.csv", submission, delimiter=",", fmt='%s')

100%|██████████| 50/50 [00:03<00:00, 14.08it/s]
